## Hyperparameter Transfer

In the following example, we aim to find a single configuration 
from which we hope to achieve optimal performance on future datasets.

In this case, we are interested in a solution across `instances`. 
We therefore do **not** set the instance on construction.

In [38]:
from yahpo_gym import * 
import random
b = BenchmarkSet("lcbench")

In [34]:
# First we split  the instances (= datasets) into train and  test instances
train_ins = random.sample(b.instances, 20)
test_ins = [x for x in b.instances if x not in train_ins]

In [44]:
def batchify(xs, instances, ins_name):
    """
    Turn a single configuration (Dictionary) into a list of configurations with varying instances.
    """
    res = []
    for idx in instances:
        xc = xs.copy()
        xc.update({ins_name: idx})
        res += [xc]
    return res

def draw_batch(instances, ins_name):
    """
    Draw a random configuration across all instances
    """
    cs = b.get_opt_space().sample_configuration(1).get_dictionary()
    return batchify(cs,instances, ins_name)

In [46]:
# FIXME: Waiting for new surrogates
# bb = draw_batch(train_ins, 'OpenML_task_id')
# b.objective_function(bb)